In [25]:
import h5py
import torch
from src.data.play import Play, slice_keys
from src.utilities.embedding_utilities import n_players
from torch.nn import Embedding
from torch import Tensor as t

In [26]:
print(slice_keys)

{'offense_roster': slice(2, 7, None), 'defense_roster': slice(7, 12, None)}


In [27]:
db = h5py.File("cache/full_db.h5", "r")

In [28]:
embedding_dim = 2
embeds = Embedding(n_players, embedding_dim)

In [33]:
i = 0
lst = []
for p in db["2016_regular/plays"][0:10]:
    p = t(p)
    lst.append(p)
p = torch.stack(lst)
print(p.shape)
offense_indices = p[:, slice_keys['offense_roster']].long()
defense_indices = p[:, slice_keys['defense_roster']].long()
embeddings = embeds(offense_indices)
print(embeddings.shape)

torch.Size([10, 42])
torch.Size([10, 5, 2])


In [45]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch import nn
from pytorch_lightning import Trainer, LightningModule

class LitMNIST(LightningModule):
    def __init__(self):
        super().__init__()

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)

        x = F.log_softmax(x, dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

    def train_dataloader(self):
        # transforms
        # prepare transforms standard to MNIST
        transform=transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))])
        # data
        mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
        return DataLoader(mnist_train, batch_size=64, num_workers=8)

In [44]:
model = LitMNIST()
trainer = Trainer()
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


NameError: name 'Adam' is not defined